# Import modules

In [ ]:
import os
import pandas as pd


import numpy as np
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import power_transform
from catboost import CatBoostRegressor

# Local dependencies
from src.constants import (
    PREPROCESSING_PATH,
    TARGET
)
from utils.classes.extractor import FunctionalGroupToGramExtractor

from typing import (
    Optional,
    List,
    Any
)
from sklearn.model_selection import KFold, RepeatedKFold, RepeatedStratifiedKFold

import pickle
import json

import warnings
warnings.filterwarnings("ignore")

# Load essentials

In [ ]:
Train = pd.read_csv(f"{PREPROCESSING_PATH}/train_merged_CIF/Train.csv")
Train = Train.set_index(Train.columns[0])

Pretest = pd.read_csv(f"{PREPROCESSING_PATH}/pretest_merged_CIF/Pretest.csv")
Pretest = Pretest.set_index(Pretest.columns[0])

funtional_group_extractor = FunctionalGroupToGramExtractor()

In [ ]:
Train['surface_area [m^2/g]']

Unnamed: 0
1           0.00
2         603.61
3         788.50
4        1441.53
5           0.00
          ...   
68609       0.00
68610       0.00
68611       0.00
68612       0.00
68613       0.00
Name: surface_area [m^2/g], Length: 68613, dtype: float64

# Add stat

In [ ]:
# stat = pd.read_csv("/work/preprocessing_results/stat/functionalgroup_stat2.csv")

# Train['functional_groups'] = np.where(
#     pd.isna(Train['functional_groups']),
#     'unknown',
#     Train['functional_groups']
# )

# Train = Train.merge(stat, on=['functional_groups','topology'], how = 'left')

# Pretest['functional_groups'] = np.where(
#     pd.isna(Pretest['functional_groups']),
#     'unknown',
#     Pretest['functional_groups']
# )

# Pretest = Pretest.merge(stat, on=['functional_groups','topology'], how = 'left')

# Add imputor

In [ ]:
# train_imputation = pd.read_csv("imputer/xgboost/train.csv", index_col=0).drop(
#     [
#         'MOFname',
#         'void_fraction_imputed',
#         'void_volume_imputed',
#         'surface_area_imputed',
#     ],
#     axis=1
# )
# # Train = Train.rename(columns={'surface_area [m^2/g]': 'surface_area_old'})
# Train = pd.concat(
#     [
#         Train.drop([
#             'void_fraction',
#             'void_volume [cm^3/g]',
#             'surface_area [m^2/g]'
#         ], axis=1),
#         train_imputation
#     ], axis=1
# )

# pretest_imputation = pd.read_csv("imputer/xgboost/pretest.csv", index_col=0).drop(
#     [
#         'MOFname',
#         'void_fraction_imputed',
#         'void_volume_imputed',
#         'surface_area_imputed'
#     ],
#     axis=1
# )
# Pretest = pd.concat(
#     [
#         Pretest.drop([
#             'void_fraction',
#             'void_volume [cm^3/g]',
#             'surface_area [m^2/g]'
#         ], axis=1),
#         pretest_imputation
#     ], axis=1
# )

# Set up pipeline

In [ ]:

class PreprocessingPipeline:
    funcgroup2num = None
    topology2num = None
    spacegroup2num = None
    organic_linker1 = None
    organic_linker2 = None

    def label(self, df):
        # if self.funcgroup2num is None:
        #     self.funcgroup2num = {
        #         c: i
        #         for i, c
        #         in enumerate(df["functional_groups"].unique())
        #     }

        # if self.topology2num is None:
        #     self.topology2num = {
        #         c: i
        #         for i, c
        #         in enumerate(df["topology"].unique())
        #     }
        # df["label_topology"] = df["topology"].map(self.topology2num)

        df["label_topology_pcu"] = (df["topology"] == "pcu").astype(int)
        df["label_topology_sra"] = (df["topology"] == "sra").astype(int)
        df["label_topology_acs"] = (df["topology"] == "acs").astype(int)
        df["label_topology_etb"] = (df["topology"] == "etb").astype(int)
        df["label_topology_bcu"] = (df["topology"] == "bcu").astype(int)
        df["label_topology_nbo"] = (df["topology"] == "nbo").astype(int)
        
        
        df["label_spacegroup_triclinic"] = (
            df["_space_group_crystal_system"] == "triclinic"
        ).astype(int)

        df["metal_linker_1"] = (
            df["metal_linker"] == 1
        ).astype(int)
        df["metal_linker_2"] = (
            df["metal_linker"] == 2
        ).astype(int)
        df["metal_linker_3"] = (
            df["metal_linker"] == 3
        ).astype(int)
        df["metal_linker_4"] = (
            df["metal_linker"] == 4
        ).astype(int)
        df["metal_linker_9"] = (
            df["metal_linker"] == 9
        ).astype(int)
        df["metal_linker_10"] = (
            df["metal_linker"] == 10
        ).astype(int)
        df["metal_linker_12"] = (
            df["metal_linker"] == 12
        ).astype(int)

        if self.organic_linker1 is None:
            self.organic_linker1 = set(df['organic_linker1'])
        for linker in self.organic_linker1:
            df[f"organic_linker1_{linker}"] = (
                df["organic_linker1"] == linker
            ).astype(int)

        if self.organic_linker2 is None:
            self.organic_linker2 = set(df['organic_linker2'])
        for linker in self.organic_linker2:
            df[f"organic_linker2_{linker}"] = (
                df["organic_linker2"] == linker
            ).astype(int)

        return df
    

    @staticmethod
    def get_density(df: pd.DataFrame):
        df['density'] = (df["weight [u]"] / df["volume [A^3]"]) * 1.66054
        df['void_volume [cm^3/g]'] = df['void_fraction'] / df["density"]
        return df

    @staticmethod
    def replace_surface_area_equal_0_with_null(df: pd.DataFrame):
        df["surface_area [m^2/g]"] = df["surface_area [m^2/g]"].replace(0, np.nan)
        return df

    @staticmethod
    def replace_inf_with_null(df: pd.DataFrame):
        df = df.replace(np.inf, 999999)
        df = df.replace(-np.inf, -999999)
        return df

    @staticmethod
    def drop_unused_columns(
        df, unused_columns: Optional[List[str]] = None
    ) -> pd.DataFrame:
        if not unused_columns:
            unused_columns = [
                "MOFname",
                # 'functional_groups',
                "topology",
                "cif_filepath",
                "_audit_creation_date",
                "_symmetry_Int_Tables_number",
                "_symmetry_space_group_name_H-M",
                "_space_group_crystal_system",
                # "bond_type_countT",
                'metal_linker',
                # 'organic_linker1',
                # 'organic_linker2'
                "partial_charge_mean",
                "partial_charge_std",
                "_cell_volume",
                'mof_convex_hull_area',
                'mof_convex_hull_volume',
                # 'mof_convex_hull_npoints',
                # 'mof_convex_hull_nsimplex'
            ]
        for col in unused_columns:
            try:
                df.drop(col, axis=1, inplace=True)
            except KeyError:
                pass
        return df

    @staticmethod
    def set_imputer(X: pd.DataFrame) -> KNNImputer:
        imputer = KNNImputer(n_neighbors=5)
        imputer.fit(X)
        return imputer

    @staticmethod
    def impute_value(X: pd.DataFrame, imputer: KNNImputer):
        return imputer.transform(X)

    @staticmethod
    def extract_functional_group(X: pd.DataFrame, fit: bool) -> pd.DataFrame:
        funtional_group_extacted = funtional_group_extractor.transform(X, fit)
        return pd.concat([X, funtional_group_extractor], axis=1)


class TrainDataPreprocessingPipeline(PreprocessingPipeline):
    @staticmethod
    def drop_surface_area_equal_minus_1(df):
        return df.drop(df[df["surface_area [m^2/g]"] == -1].index)

    def impute(self):
        temp_columns = self.X.columns
        temp_index = self.X.index

        if not self.imputer:
            self.imputer = self.set_imputer(self.X)

        self.X = self.impute_value(self.X, self.imputer)

        self.X = pd.DataFrame(self.X)
        self.X.columns = temp_columns
        self.X.index = temp_index

    def run(self):
        self.functional_group_extractor = FunctionalGroupToGramExtractor()

        # Drop and add featrures
        print("Print droping and replace null")
        self.df = self.drop_surface_area_equal_minus_1(self.df)
        self.df = self.replace_surface_area_equal_0_with_null(self.df)
        self.df = self.replace_inf_with_null(self.df)
        self.df = self.label(self.df)
        self.df = self.drop_unused_columns(self.df)
        self.df = self.df.drop(self.df[self.df["CO2/N2_selectivity"] == 0].index)
        self.df = self.get_density(self.df)
        # Split
        print("Split X, y")
        self.X = self.df.drop(TARGET, axis=1)
        self.y = self.df[[TARGET]]

        # Extract
        # print("Extract functional group")
        # functional_group = self.functional_group_extractor.transform(
        #     self.X[['functional_groups']],
        #     fit=True
        # )

        # Impute
        print("Impute")
        self.X = self.X.drop("functional_groups", axis=1)
        self.impute()
        # self.X = pd.concat([self.X, functional_group], axis=1)

    def __init__(self, df: pd.DataFrame, imputer: Any = None):
        self.df = df
        self.imputer = imputer


In [ ]:
class TestDataPreprocessingPipeline(PreprocessingPipeline):
    def impute(self):
        temp_columns = self.X.columns
        temp_index = self.X.index

        if not self.imputer:
            self.imputer = self.set_imputer(self.X)

        self.X = self.impute_value(self.X, self.imputer)

        self.X = pd.DataFrame(self.X)
        self.X.columns = temp_columns
        self.X.index = temp_index

    def run(self):
        # Drop
        print("Print droping and replace null")
        self.X = self.replace_surface_area_equal_0_with_null(self.df)
        self.X = self.replace_inf_with_null(self.X)
        self.X = self.label(self.X)
        self.X = self.drop_unused_columns(self.X)
        self.X = self.get_density(self.X)

        # Extract
        # print("Extract functional group")
        # functional_group = self.functional_group_extractor.transform(
        #     self.X[['functional_groups']],
        # )

        # Impute
        print("Impute")
        self.X = self.X.drop('functional_groups', axis=1)
        self.impute()
        # self.X = pd.concat([self.X, functional_group], axis=1)
        self.X = self.X[self.columns]

    
    def __init__(
        self,
        df: pd.DataFrame,
        imputer: Any = None,
        functional_group_extractor: FunctionalGroupToGramExtractor = None,
        columns: list = None
    ):
        self.df = df
        self.imputer = imputer
        self.functional_group_extractor = functional_group_extractor
        self.columns = columns



# Run pipeline

In [ ]:
class Imputer9999:
    def transform(self, df):
        return df.replace(np.nan, -9999)

imputer = Imputer9999()
train = TrainDataPreprocessingPipeline(Train, imputer)
train.run()

Print droping and replace null
Split X, y
Impute


In [ ]:
train.X

,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,void_volume [cm^3/g],organic_linker1,organic_linker2,CO2/N2_selectivity,heat_adsorption_CO2_P0.15bar_T298K [kcal/mol],_cell_length_a,...,organic_linker2_51,organic_linker2_52,organic_linker2_53,organic_linker2_54,organic_linker2_55,organic_linker2_56,organic_linker2_57,organic_linker2_58,organic_linker2_59,density
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
1,1116.667429,875.240600,-9999.00,0.07899,0.060690,4,11,22.864166,6.786041,10.609882,...,0,0,0,0,0,0,0,0,0,1.301526
2,2769.503842,2211.697211,603.61,0.13794,0.104020,44,57,33.616780,7.147286,8.463295,...,0,0,0,0,0,0,1,0,0,1.326090
3,1089.818728,773.687960,788.50,0.14874,0.126173,22,24,19.263726,6.347967,10.732110,...,0,0,0,0,0,0,0,0,0,1.178856
4,2205.198301,1304.638720,1441.53,0.21814,0.222046,17,24,25.701377,6.190085,6.935530,...,0,0,0,0,0,0,0,0,0,0.982408
5,1137.800963,901.736120,-9999.00,0.07778,0.059102,1,22,30.001838,6.478063,10.825925,...,0,0,0,0,0,0,0,0,0,1.316020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68609,1188.302573,1001.700216,-9999.00,0.00000,0.000000,4,24,24.131770,-9999.000000,10.718161,...,0,0,0,0,0,0,0,0,0,1.399781
68610,1506.660363,1493.296496,-9999.00,0.01108,0.006732,42,46,6.071818,-9999.000000,8.192620,...,0,0,0,0,0,0,0,0,0,1.645811
68611,2035.532738,1959.518320,-9999.00,0.00000,0.000000,14,22,9.876134,-9999.000000,11.237482,...,0,0,0,0,0,0,0,0,0,1.598529


In [ ]:
test = TestDataPreprocessingPipeline(
    Pretest,
    train.imputer,
    train.functional_group_extractor,
    train.X.columns
)
test.organic_linker1 = train.organic_linker1
test.organic_linker2 = train.organic_linker2
test.topology2num = train.topology2num
test.spacegroup2num = train.spacegroup2num
test.run()

Print droping and replace null
Impute


In [ ]:
assert all(train.X.columns == test.X.columns)

In [ ]:
len(train.X.columns)

152

In [ ]:
1/test.X[train.X.columns][['density']]

,density
Unnamed: 0,
1,1.666998
2,1.214611
3,1.141007
4,1.543144
5,0.660375
...,...
1996,1.950456
1997,1.140249
1998,2.262239


In [ ]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import (
    VotingRegressor,
    StackingRegressor,
    RandomForestRegressor,
    GradientBoostingRegressor
)
from sklearn.linear_model import Lasso

In [ ]:
class RmseMetric(object):
    def get_final_error(self, error, weight):
        return np.sqrt(error / (weight + 1e-38))

    def is_max_optimal(self):
        return False

    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])

        approx = approxes[0]

        error_sum = 0.0
        weight_sum = 0.0

        for i in range(len(approx)):
            w = 1.0 if weight is None else weight[i]
            weight_sum += w
            error_sum += w * ((approx[i] - target[i])**2)

        return error_sum, weight_sum

In [ ]:
def log_mean_absolute_error(y_true, y_pred):
    return np.log(mean_absolute_error(y_true, y_pred))


def fit_catboost(X, y):
    catboost = CatBoostRegressor(
        iterations=5000,
        verbose=False,
        # loss_function="MAE"
        # l2_leaf_reg=0.001
    )
    catboost.fit(X, y)
    return catboost


def fit_xgboost(X, y):
    reg = XGBRegressor()
    reg.fit(X.values, y.values)
    return reg


def fit_lightboost(X, y):
    reg = LGBMRegressor()
    reg.fit(X.values, y.values)
    return reg


def fit_voting(X, y):
    reg = VotingRegressor(
        [
            ('cat', CatBoostRegressor(
                    iterations=5000,
                    # iterations=500,
                    grow_policy='Lossguide',
                    verbose=False,
                    # depth=14
                    # l2_leaf_reg=0.001
                )
            ),
            ('xgb', XGBRegressor(
                    grow_policy='lossguide',
                    # tree_method='exact'
                    max_depth=14
                    # max_depth=6
                )
            ),
            (
                'light', LGBMRegressor(
                    num_iterations=5000
                )
            )
            # ('gb', GradientBoostingRegressor())
        ]
    )
    reg.fit(X.values, y.values)
    return reg


fit_models = [fit_voting]
# kf = KFold(n_splits=5, shuffle=True, random_state=1234)
kf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=1234)
# kf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=1234)
fit_results = {}
for k, (train_index, test_index) in enumerate(kf.split(train.X)):
    print(f"K Fold: {k + 1}")
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = train.X.iloc[train_index], train.X.iloc[test_index]
    y_train, y_test = train.y.iloc[train_index], train.y.iloc[test_index]
    
    for fit_model in fit_models:
        model_name = '_'.join(fit_model.__name__.split('fit_')[1:])
        model_result_path = f"model_results/{model_name}"
        model_checkpoint = f"model_results/{model_name}/fold_{k+1}.pickle"
        
        if not os.path.exists(model_result_path):
            os.makedirs(model_result_path)
        if model_name not in fit_results:
            fit_results[model_name] = []
        if os.path.isfile(model_checkpoint):
            print("Found trained model")
            # continue
            with open(model_checkpoint, 'rb') as f:
                try:
                    model = pickle.load(f)
                    print("Open successfully")
                except EOFError:
                    print("Open error")
                    model = fit_model(X_train, y_train)
                    with open(model_checkpoint, "wb") as f:
                        pickle.dump(model, f)

        else:
            model = fit_model(X_train, y_train)
            with open(model_checkpoint, "wb") as f:
                pickle.dump(model, f)

        train_pred = model.predict(X_train)
        log_mean_error_train = log_mean_absolute_error(y_train, train_pred)

        # model = fit_model(X_test, y_test)
        test_pred = model.predict(X_test)
        log_mean_error_test = log_mean_absolute_error(y_test, test_pred)

        # test_pred = pd.DataFrame(
        #     test_pred,
        #     index= y_test.index,
        #     columns=y_test.columns
        # )
        # error_sort = np.abs(y_test - test_pred).sort_values(TARGET, ascending=False)
        

        print(f"Log mean error train: {log_mean_error_train}")
        print(f"Log mean error test: {log_mean_error_test}")
        # print(error_sort.head(10))
        
        # with open(f"model_results/{model_name}/fold_{k+1}.pickle", "wb") as f:
        #     pickle.dump(model, f)
        
        fit_results[model_name].append({
            'model': model,
            'log_mean_error_train': log_mean_error_train,
            'log_mean_error_test': log_mean_error_test
        })

K Fold: 1
TRAIN: [    1     2     3 ... 66712 66713 66714] TEST: [    0     4    13 ... 66699 66708 66711]
Found trained model
Open successfully
Log mean error train: 1.9021937648164438
Log mean error test: 2.9507346569510866
K Fold: 2
TRAIN: [    0     1     2 ... 66711 66713 66714] TEST: [   18    24    30 ... 66700 66703 66712]
Found trained model
Open successfully
Log mean error train: 1.8900964142859384
Log mean error test: 2.9585282582174584
K Fold: 3
TRAIN: [    0     1     3 ... 66712 66713 66714] TEST: [    2     8     9 ... 66689 66695 66710]
Found trained model
Open successfully
Log mean error train: 1.905114527954128
Log mean error test: 2.961797079296942
K Fold: 4
TRAIN: [    0     2     3 ... 66711 66712 66713] TEST: [    1     5     6 ... 66705 66709 66714]
Found trained model
Open successfully
Log mean error train: 1.8933862340827703
Log mean error test: 2.983269042431703
K Fold: 5
TRAIN: [    0     1     2 ... 66711 66712 66714] TEST: [    3    10    16 ... 66706 66707

Open successfully
Log mean error train: 1.893863817889665
Log mean error test: 2.971183437673779
K Fold: 38
TRAIN: [    0     1     2 ... 66712 66713 66714] TEST: [    8     9    12 ... 66698 66706 66707]
Found trained model
Open successfully
Log mean error train: 1.8939856999931377
Log mean error test: 2.9626740849599043
K Fold: 39
TRAIN: [    0     2     3 ... 66710 66713 66714] TEST: [    1     7    14 ... 66708 66711 66712]
Found trained model
Open successfully
Log mean error train: 1.895794650514479
Log mean error test: 2.9633059871897856
K Fold: 40
TRAIN: [    0     1     2 ... 66712 66713 66714] TEST: [    3     6    10 ... 66694 66702 66704]
Found trained model
Open successfully
Log mean error train: 1.9147390258482373
Log mean error test: 2.956326006001331
K Fold: 41
TRAIN: [    0     3     4 ... 66711 66713 66714] TEST: [    1     2     8 ... 66705 66709 66712]
Found trained model
Open successfully
Log mean error train: 1.897929453986147
Log mean error test: 2.974838562916848

In [ ]:
"""
funcgroup2num = {c: i for i, c in enumerate(train['functional_groups'].unique())}
topology2num = {c: i for i, c in enumerate(train['topology'].unique())}
spacegroup2num = {c: i for i, c in enumerate(train['_space_group_crystal_system'].unique())}

train['label_funcgroup'] = train['functional_groups'].map(funcgroup2num)
train['label_topology'] = train['topology'].map(topology2num)
train['label_spacegroup'] = train['_space_group_crystal_system'].map(spacegroup2num)
"""

"\nfuncgroup2num = {c: i for i, c in enumerate(train['functional_groups'].unique())}\ntopology2num = {c: i for i, c in enumerate(train['topology'].unique())}\nspacegroup2num = {c: i for i, c in enumerate(train['_space_group_crystal_system'].unique())}\n\ntrain['label_funcgroup'] = train['functional_groups'].map(funcgroup2num)\ntrain['label_topology'] = train['topology'].map(topology2num)\ntrain['label_spacegroup'] = train['_space_group_crystal_system'].map(spacegroup2num)\n"

In [ ]:
# np.mean([score['log_mean_error_test'] for score in fit_results['voting']])
np.mean([score['log_mean_error_test'] for score in fit_results[model_name]])

2.9655183169011696

In [ ]:
np.std([score['log_mean_error_test'] for score in fit_results[model_name]])

0.008864185749900933

In [ ]:
test_pred = pd.DataFrame(test_pred, index= y_test.index, columns=y_test.columns)
error_sort = np.abs(y_test - test_pred).sort_values(TARGET, ascending=False)

In [ ]:
Train.loc[error_sort.index].head(500).mean()

volume [A^3]                                     2.283524e+03
weight [u]                                       1.481036e+03
surface_area [m^2/g]                             6.624606e+02
void_fraction                                    1.470475e-01
void_volume [cm^3/g]                             1.470400e-01
metal_linker                                     4.954000e+00
organic_linker1                                  1.049200e+01
organic_linker2                                  1.898600e+01
CO2/N2_selectivity                               5.049219e+01
heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]    7.419032e+00
_symmetry_Int_Tables_number                      1.000000e+00
_cell_length_a                                   1.436595e+01
_cell_length_b                                   1.189161e+01
_cell_length_c                                   1.505748e+01
_cell_angle_alpha                                9.119279e+01
_cell_angle_beta                                 9.194879e+01
_cell_an

In [ ]:
error_train = Train.loc[error_sort.index]
Train["LMAE"] = error_sort.iloc[:, 0]
Train["prediction"] = test_pred.iloc[:, 0]

In [ ]:
Train.loc[error_sort.index].to_csv("error_train.csv")

In [ ]:
Train.loc[error_sort.index].tail(500).mean()

volume [A^3]                                     6.261920e+03
weight [u]                                       2.060874e+03
surface_area [m^2/g]                             2.704232e+03
void_fraction                                    4.048140e-01
void_volume [cm^3/g]                             7.856454e-01
metal_linker                                     3.482000e+00
organic_linker1                                  1.106600e+01
organic_linker2                                  2.043000e+01
CO2/N2_selectivity                               1.704419e+01
heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]    4.881901e+00
_symmetry_Int_Tables_number                      1.000000e+00
_cell_length_a                                   1.697690e+01
_cell_length_b                                   1.671859e+01
_cell_length_c                                   1.838434e+01
_cell_angle_alpha                                9.318542e+01
_cell_angle_beta                                 9.477482e+01
_cell_an

In [ ]:
# train_prediction = .predict(X)

In [ ]:
# np.log(mean_absolute_error(train_prediction, y))

In [ ]:
# feature_importances = pd.DataFrame(catboost.feature_importances_)
# feature_importances.index = Train.drop(TARGET, axis=1).columns

In [ ]:
# feature_importances.sort_values(by=0, ascending=False)

In [ ]:
# feature_importances[0].nlargest(20).plot(kind='barh')

In [ ]:
# feature_importances[0].nlargest(20).index

In [ ]:
sub = np.mean(
    [model['model'].predict(test.X) for model in fit_results[model_name]],
    axis=0
)

In [ ]:
sub = pd.DataFrame(sub)

In [ ]:
# Pretest = pd.read_csv(f"{PREPROCESSING_PATH}/pretest_merged_CIF/Pretest.csv")
# Pretest = Pretest.set_index(Pretest.columns[0])
sub.index = "pretest_" + pd.Index(range(1, sub.shape[0] + 1)).astype(str)
sub.index = sub.index.set_names('id')
sub.columns = ['CO2_working_capacity [mL/g]']

In [ ]:
sub

,CO2_working_capacity [mL/g]
id,
pretest_1,107.802950
pretest_2,127.749714
pretest_3,195.325979
pretest_4,63.653623
pretest_5,95.363564
...,...
pretest_1996,1.177091
pretest_1997,3.387031
pretest_1998,-0.318165


In [ ]:
sub.to_csv("submission.csv")

In [ ]:
import zipfile
import hashlib
 

sha256_hash = hashlib.sha256()
with open("submission.csv","rb") as f:
    # Read and update hash string value in blocks of 4K
    for byte_block in iter(lambda: f.read(4096),b""):
        sha256_hash.update(byte_block)
    hash_str = sha256_hash.hexdigest()

print(hash_str)
zipfile.ZipFile(f'{hash_str}.zip', mode='w').write("submission.csv")

c3ec960477e07bbd27988651d02c5b4377a74285d87701a3842a64dad968e917


In [ ]:
# !rm -rf *.zip

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e59b976c-8525-4130-bc49-8505c94328e0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>